In [26]:
# --- IMPORTS NECESARIOS ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Suponiendo que df_master ya está cargado. Si no, descomenta la siguiente línea:
df_master = pd.read_csv('../01_data/output/master_df.csv') 
# (Asegúrate de que la ruta sea correcta desde tu ubicación actual)

print("Inicializando modelo XGBoost y variables de prueba...")

# 1. Preparación de Variables (One-Hot Encoding para XGBoost)
# Se replican los pasos de la Fase 3
df_model_full = pd.get_dummies(df_master.drop(columns=['InvoiceDate', 'Local_Currency']), 
                               columns=['StockCode', 'Country'])

# 2. Definición de Features (X) y Target (Y)
X = df_model_full.drop(columns=['Total_Quantity_Sold'])
Y = df_model_full['Total_Quantity_Sold']

# 3. División Entrenamiento/Prueba
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42 # Usamos el mismo random_state para consistencia
)

# 4. Re-entrenar el Modelo (Define xgb_model)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, Y_train)

# Verificación de Precisión (Opcional, pero bueno tenerla)
Y_pred = xgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print("✅ Modelo XGBoost re-entrenado y X_test/Y_test cargados.")
print(f"   RMSE (Error de Predicción): {rmse:.2f}")

Inicializando modelo XGBoost y variables de prueba...
✅ Modelo XGBoost re-entrenado y X_test/Y_test cargados.
   RMSE (Error de Predicción): 13.11


In [27]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd 
import numpy as np # <-- Necesario para np.log

# --- PREPROCESAMIENTO ---
df_master['InvoiceDate'] = pd.to_datetime(df_master['InvoiceDate'])

# 1. Cálculo de RFM
SNAPSHOT_DATE = df_master['InvoiceDate'].max() + pd.Timedelta(days=1)
df_rfm = df_master.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (SNAPSHOT_DATE - x.max()).days),
    Frequency=('InvoiceDate', 'nunique'),
    Monetary=('Total_Quantity_Sold', 'sum') 
).reset_index()

# --- 1.5 TRANSFORMACIÓN LOGARÍTMICA (PASO CLAVE PARA K-MEANS) ---
# Se aplica a todas las métricas, especialmente Frequency y Monetary, para reducir el sesgo.
df_rfm['Recency_Log'] = np.log(df_rfm['Recency'] + 1)
df_rfm['Frequency_Log'] = np.log(df_rfm['Frequency'] + 1)
df_rfm['Monetary_Log'] = np.log(df_rfm['Monetary'] + 1)

# 2. Segmentación con Cuartiles (Puntuación simple)
# Se usa la métrica original para las puntuaciones, ya que queremos puntuar los valores reales.

# R_Score: Se mantiene en 5 cuartiles
df_rfm['R_Score'] = pd.qcut(
    df_rfm['Recency'], 
    5, 
    labels=[5, 4, 3, 2, 1],
    duplicates='drop'
) 

# F_Score: Usa pd.cut con 3 grupos fijos por el sesgo extremo (solución al ValueError)
min_freq = df_rfm['Frequency'].min()
max_freq = df_rfm['Frequency'].max()
bins_f = [min_freq - 1, 1, 3, max_freq] 
labels_f = [1, 2, 3] 

df_rfm['F_Score'] = pd.cut(
    df_rfm['Frequency'], 
    bins=bins_f, 
    labels=labels_f,
    include_lowest=True,
    right=True
)

# M_Score: 3 cuartiles (terciles) para consistencia
df_rfm['M_Score'] = pd.qcut(
    df_rfm['Monetary'], 
    3, 
    labels=[1, 2, 3], 
    duplicates='drop'
)

df_rfm['RFM_Score'] = df_rfm['R_Score'].astype(str) + df_rfm['F_Score'].astype(str) + df_rfm['M_Score'].astype(str)

# 3. Clustering K-Means (Opción 2: ML Avanzado)
# Escalar datos para K-Means
# ¡IMPORTANTE! Usamos las columnas _Log para el clustering.
X_rfm = df_rfm[['Recency_Log', 'Frequency_Log', 'Monetary_Log']] 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_rfm)

# Aplicar K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_rfm['Cluster'] = kmeans.fit_predict(X_scaled)

# 4. Interpretación del Clustering
# Para interpretar los clústeres, usamos las métricas ORIGINALES (sin el log)
# ya que representan el valor real del negocio.
cluster_profile = df_rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()
print("\n--- PERFIL DE LOS CLUSTERS DE CLIENTES (RFM) ---")
print(cluster_profile)


--- PERFIL DE LOS CLUSTERS DE CLIENTES (RFM) ---
            Recency  Frequency    Monetary
Cluster                                   
0         14.434708   9.618557  701.835052
1        171.679466   1.158598   27.923205
2         16.837877   1.807747   37.793400
3         75.125133   3.018028  160.952280


In [28]:
# Muestra los 10 valores de frecuencia más comunes
print(df_rfm['Frequency'].value_counts().head(10))
# Muestra los percentiles para ver los cortes
print(df_rfm['Frequency'].quantile([0.25, 0.5, 0.75, 1.0]))

Frequency
1     1463
2      616
3      443
4      260
5      170
6      119
7       67
8       59
10      37
9       36
Name: count, dtype: int64
0.25      1.0
0.50      2.0
0.75      4.0
1.00    140.0
Name: Frequency, dtype: float64


In [29]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd 

# --- PREPROCESAMIENTO ---
df_master['InvoiceDate'] = pd.to_datetime(df_master['InvoiceDate'])

# 1. Cálculo de RFM
SNAPSHOT_DATE = df_master['InvoiceDate'].max() + pd.Timedelta(days=1)
df_rfm = df_master.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (SNAPSHOT_DATE - x.max()).days),
    Frequency=('InvoiceDate', 'nunique'),
    Monetary=('Total_Quantity_Sold', 'sum') 
).reset_index()

# 2. Segmentación con Cuartiles (Opción 1: Corrección Final)



In [30]:
# Definir el Costo (Reutilizamos el costo fijo que ya calculaste)
average_price = df_master['UnitPrice'].mean()
FIXED_COST = average_price * 0.50 

def find_optimal_price_for_row(row, xgb_model, price_steps=50):
    """
    Busca el precio óptimo para una única fila de datos (Producto/País/Día).
    Devuelve el precio óptimo y la ganancia máxima esperada.
    """
    best_profit = -np.inf
    optimal_price = row['UnitPrice'] # Inicializamos con el precio actual

    # Clonamos la fila y la convertimos en un DataFrame con una sola fila
    # Esto es necesario porque el modelo XGBoost espera un DataFrame
    features_to_predict = pd.DataFrame([row], columns=row.index)
    
    # Definir el rango de precios a probar (Ej. +/- 20% del precio actual)
    current_price = row['UnitPrice']
    price_range = np.linspace(
        current_price * 0.8, 
        current_price * 1.2, 
        num=price_steps
    )
    
    # 2. Bucle de Optimización (Prueba y Error)
    for price in price_range:
        # Actualizar el precio propio en las features clonadas
        features_to_predict.loc[:, 'UnitPrice'] = price
        
        # Predecir la Demanda con el nuevo precio
        # Usamos el modelo previamente entrenado (xgb_model)
        predicted_demand = xgb_model.predict(features_to_predict)[0]
        
        # Calcular la Ganancia: (Precio - Costo) * Demanda
        profit = (price - FIXED_COST) * predicted_demand
        
        # Actualizar el precio óptimo si se encuentra una mayor ganancia
        if profit > best_profit:
            best_profit = profit
            optimal_price = price
            
    return optimal_price, best_profit

In [31]:
# --- APLICACIÓN MASIVA DEL MOTOR PRESCRIPTIVO ---

# 1. Seleccionar un subconjunto de datos de prueba para la demo masiva
# Usamos X_test (que contiene las features One-Hot Encoded)
SAMPLE_SIZE = 500
df_recommendations = X_test.head(SAMPLE_SIZE).copy()

print(f"Generando {SAMPLE_SIZE} recomendaciones de precios...")

# 2. Aplicar la función a cada fila y generar las nuevas columnas
# Usamos un bucle for simple para aplicar la función a cada fila.
results = df_recommendations.apply(
    lambda row: find_optimal_price_for_row(row, xgb_model), 
    axis=1, result_type='expand'
)

results.columns = ['Optimal_Price', 'Max_Profit']

# 3. Combinar los resultados con las características originales
df_recommendations = pd.concat([df_recommendations, results], axis=1)

# 4. Mapear las columnas clave para la visualización
# Necesitamos mapear de vuelta a StockCode y Country para el reporte
def get_original_id(row, prefix):
    # Función para revertir el One-Hot Encoding y obtener el nombre original
    cols = [col for col in row.index if col.startswith(prefix) and row[col] == 1]
    return cols[0].split('_')[-1] if cols else 'N/A'

df_recommendations['StockCode'] = df_recommendations.apply(lambda row: get_original_id(row, 'StockCode'), axis=1)
df_recommendations['Country'] = df_recommendations.apply(lambda row: get_original_id(row, 'Country'), axis=1)


# 5. Cálculo del Impacto Financiero
df_recommendations['Current_Profit'] = (df_recommendations['UnitPrice'] - FIXED_COST) * Y_test.head(SAMPLE_SIZE)
df_recommendations['Profit_Uplift_Pct'] = ((df_recommendations['Max_Profit'] - df_recommendations['Current_Profit']) / df_recommendations['Current_Profit']) * 100

# 6. Presentación de la Tabla de Recomendaciones
report_cols = ['StockCode', 'Country', 'UnitPrice', 'Optimal_Price', 
               'Current_Profit', 'Max_Profit', 'Profit_Uplift_Pct']

print("\n" + "="*80)
print("✅ TABLA DE PRECIOS RECOMENDADOS (MOTOR PRESCRIPTIVO A GRAN ESCALA)")
print("Las recomendaciones están basadas en maximizar la Ganancia por Transacción.")
print("="*80)
print(df_recommendations[report_cols].sort_values(by='Profit_Uplift_Pct', ascending=False).head(10).to_markdown(index=False, floatfmt=".2f"))
print("\n" + "="*80)
print(f"Mejora Promedio de Ganancia en la muestra: {df_recommendations['Profit_Uplift_Pct'].mean():.2f}%")

Generando 500 recomendaciones de precios...


KeyboardInterrupt: 